In [8]:
using Interpolations
using Distributed
using DataFrames
using Printf
using JLD
using CSV
using Dierckx
using Dates
using PyPlot
# using PyCall
using Seaborn
using LaTeXStrings

main_path = "/home/artur/BondPricing"
module_path = string(main_path, "/", "Julia/modules/")
modls = ["Batch", "ModelObj",
         "AnalyticFunctions", "BondPrInterp",
         "EqFinDiff", "FullInfoEq",
         "ModelPlots", "JointEqStructs", "JointEq"]
for modl in modls
    include(string(joinpath(module_path, modl), ".jl"))
end

ENV["LINES"] = 750
ENV["COLUMNS"] = 10000

Figure(PyObject <Figure size 400x100 with 1 Axes>)

10000

In [3]:
# JointEq.get_jeq_mus_fname(jks_fpath; eq_type="full_info")

"/mnt/ide0/home/artur/BondPricing/Julia/Results/JEQ/kappa_25.00_bp__sigmal_0.150/m_1.00_pcr_12.00/fi_tmp.csv"

In [5]:
jks_file = "/mnt/ide0/home/artur/BondPricing/Julia/Batch/new_jks/jeq_calculator.jl"
include(jks_file)

load_df: true
rerun_fi: false
rerun_misrep: false
rerun_pool: false
rerun_sep: false
save_df: true
pool_obj_fun: exp_firm_value


Setting parameter dictionary using (m, m_comb_num) ID pair...
Loading optimal results dataframe...
Setting parameter dictionary using (m, m_comb_num) ID pair...
Setting parameter dictionary using (m, m_comb_num) ID pair...
Loading optimal results dataframe...
Setting parameter dictionary using (m, m_comb_num) ID pair...
Loading optimal results dataframe...
Setting parameter dictionary using (m, m_comb_num) ID pair...
Setting parameter dictionary using (m, m_comb_num) ID pair...
Loading optimal results dataframe...
Setting parameter dictionary using unique combination ID...
Setting initial vbl value to NaN
Setting initial vbh value to NaN
Setting initial c value to NaN
Setting initial p value to NaN
Constant Volatility Model: setting vbh to vbl, lambda to NaN
Setting parameter dictionary using unique combination ID...
Loading SVM object...
Batch object file not found.
Batch Object is incompatible! Loading surfaces only instead.
Setting initial vbl value to NaN
Setting initial vbh value 

Figure(PyObject <Figure size 400x100 with 1 Axes>)

  0.235070 seconds (811.39 k allocations: 137.079 MiB, 14.36% gc time)


(ModelObj.Firm
  mu_b: Float64 1.0
  m: Float64 1.0
  c: Float64 NaN
  p: Float64 NaN
  vbl: Float64 NaN
  vbh: Float64 NaN
  pm: ModelObj.FirmParams
  model: String "cvm"
  bi: ModelObj.BPrInputs
  bs: ModelObj.BPrSurfs
  bf: ModelObj.BPrInterpFuns
  bit: ModelObj.BPrFixedTTMInputs
  bft: ModelObj.BPrInterpFuns
  optKS: ModelObj.KStruct
, ModelObj.Firm
  mu_b: Float64 1.0
  m: Float64 1.0
  c: Float64 NaN
  p: Float64 NaN
  vbl: Float64 NaN
  vbh: Float64 NaN
  pm: ModelObj.FirmParams
  model: String "svm"
  bi: ModelObj.BPrInputs
  bs: ModelObj.BPrSurfs
  bf: ModelObj.BPrInterpFuns
  bit: ModelObj.BPrFixedTTMInputs
  bft: ModelObj.BPrInterpFuns
  optKS: ModelObj.KStruct
, Main.JointEqStructs.JointKStruct(0.2, NaN, 1.0, 4.5, 54.0, NaN, 59.82018174178425, NaN, 50.88396346568897, NaN))

In [6]:
uq_fidf = JointEqStructs.get_unique_df(jks_fpath, :fi; del_old_files=true)
uq_misrepdf = JointEqStructs.get_unique_df(jks_fpath, :misrep; del_old_files=true)
uq_pooldf = JointEqStructs.get_unique_df(jks_fpath, :pool; del_old_files=true)
uq_sepdf = JointEqStructs.get_unique_df(jks_fpath, :sep;
                                        save_filtered_df=true,
                                        del_old_files=true)

Removing old files...
Saving filtered results...
Removing old files...
Saving filtered results...
Removing old files...
Saving filtered results...
Removing old files...
Saving filtered results...


,datetime,eq_type,kappa,mu_s,m,c,p,sf_defaults_first,misrep_type,misrep_ic_objf,misrep_ic_objf_val,jeq_objf,s_MBR,s_debt,s_delta,s_eq_deriv,s_eq_deriv_min_val,s_eq_min_val,s_eq_negative,s_eq_vb,s_equity,s_firm_value,s_iota,s_lambda,s_leverage,s_mu_b,s_sigmah,s_fi_vb,st_vb,s_rmp,r_MBR,r_debt,r_delta,r_eq_deriv,r_eq_deriv_min_val,r_eq_min_val,r_eq_negative,r_eq_vb,r_equity,r_firm_value,r_iota,r_lambda,r_leverage,r_mu_b,r_sigmah,r_fi_vb,rt_vb,r_rmp,V0,alpha,pi,r,gross_delta,xi,sigmal,s_rm_iota,s_nrm_lambda,s_nrm_sigmah,r_rm_iota,r_nrm_lambda,r_nrm_sigmah
,DateTime,Symbol,Float64,Float64,Float64,Float64,Float64,Bool,Symbol,Symbol,Float64,Symbol,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Symbol,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Symbol,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2020-02-19T15:56:02.761,sep,0.0025,0.2,1.0,4.5,54.0,1,rt,MBR,15.2682,st_firm_value,30.8759,64.3026,0.02,0.000223679,0.000223679,0.000226821,0,0.0,46.7192,111.022,0.0,NaN,57.9189,1.19065,NaN,68.147,68.147,rm,15.2682,46.1807,0.02,1.09135e-5,1.09135e-5,0.0011387,0,0.0,62.0365,108.217,0.0,0.2,42.6741,0.854851,0.225,50.884,50.884,nrm,100.0,0.6,0.27,0.08,0.02,1.0,0.15,0.0,NaN,NaN,0.0,0.2,0.225
2,2020-02-20T11:54:51.359,sep,0.0025,0.2,1.0,4.5,54.0,1,rt,MBR,14.0972,st_firm_value,30.1588,61.5031,0.02,0.000224557,0.000224557,0.000247037,0,0.0,50.1071,111.61,0.0,NaN,55.1053,1.13859,NaN,65.1674,65.1674,rm,14.0972,44.6101,0.02,3.03888e-5,3.03888e-5,0.00106954,0,0.0,63.1984,107.808,0.0,0.3,41.379,0.825777,0.225,49.328,49.328,nrm,100.0,0.6,0.27,0.08,0.02,1.0,0.15,0.0,NaN,NaN,0.0,0.3,0.225
3,2020-02-20T11:55:00.22,sep,0.0025,0.2,1.0,4.5,54.0,1,rt,MBR,17.9244,st_firm_value,30.3113,67.791,0.02,0.000222583,0.000222583,0.000202307,0,0.0,41.9719,109.763,0.0,NaN,61.7613,1.25617,NaN,71.8974,71.8974,rm,17.9244,49.2423,0.02,-4.2861e-6,-4.2861e-6,0.00122897,0,0.0,59.8557,109.098,0.0,0.1,45.1359,0.911526,0.225,53.7551,53.7551,nrm,100.0,0.6,0.27,0.08,0.02,1.0,0.15,0.0,NaN,NaN,0.0,0.1,0.225
4,2020-02-20T11:55:05.962,sep,0.0025,0.2,1.0,4.5,54.0,1,rt,MBR,12.6167,st_firm_value,30.1535,61.4892,0.02,0.000224561,0.000224561,0.000247139,0,0.0,50.1232,111.612,0.0,NaN,55.0917,1.13833,NaN,65.1527,65.1527,rm,12.6167,43.2283,0.02,1.45711e-5,1.45711e-5,0.00135165,0,0.0,63.9344,107.163,0.0,0.2,40.3389,0.800199,0.25,48.0876,48.0876,nrm,100.0,0.6,0.27,0.08,0.02,1.0,0.15,0.0,NaN,NaN,0.0,0.2,0.25
5,2020-02-20T11:55:52.138,sep,0.0025,0.2,1.0,4.5,54.0,1,rt,MBR,13.5729,st_firm_value,30.91,65.4659,0.02,0.000223316,0.000223316,0.000218571,0,0.0,45.2085,110.674,0.0,NaN,59.1518,1.21238,NaN,69.391,69.391,rm,13.5729,45.3766,0.02,-4.09697e-6,-4.09697e-6,0.00179995,0,0.0,62.0374,107.414,0.0,0.1,42.2446,0.839967,0.275,50.225,50.225,nrm,100.0,0.6,0.27,0.08,0.02,1.0,0.15,0.0,NaN,NaN,0.0,0.1,0.275
6,2020-02-20T11:56:29.955,sep,0.0025,0.2,1.0,4.5,54.0,1,rt,MBR,10.5294,st_firm_value,28.5688,58.2352,0.02,0.000225607,0.000225607,0.000271296,0,0.0,53.6966,111.932,0.0,NaN,52.0274,1.07801,NaN,61.7004,61.7004,rm,10.5294,41.0243,0.02,-0.0020473,-0.0020473,0.00142121,0,0.0,65.1855,106.21,0.0,0.2,38.6257,0.759444,0.275,45.9868,45.9868,nrm,100.0,0.6,0.27,0.08,0.02,1.0,0.15,0.0,NaN,NaN,0.0,0.2,0.275
7,2020-02-20T11:56:53.859,sep,0.0025,0.2,1.0,4.5,54.0,1,rt,MBR,11.4607,st_firm_value,28.0065,57.2809,0.02,0.000225922,0.000225922,0.000278507,0,0.0,54.6832,111.964,0.0,NaN,51.16,1.06034,NaN,60.6889,60.6889,rm,11.4607,41.6855,0.02,-0.00173208,-0.00173208,0.00115478,0,0.0,64.9978,106.683,0.0,0.3,39.074,0.771635,0.25,46.5643,46.5643,nrm,100.0,0.6,0.27,0.08,0.02,1.0,0.15,0.0,NaN,NaN,0.0,0.3,0.25
8,2020-02-20T12:04:46.695,sep,0.0025,0.2,1.0,4.5,54.0,1,rt,MBR,9.2034,st_firm_value,30.4135,62.2218,0.02,0.00022433,0.00022433,0.000241797,0,0.0,49.2678,111.49,0.0,NaN,55.8095,1.15193,NaN,65.9311,65.931

In [7]:
tcdf

,st_iota,rt_iota,st_lambda,rt_lambda,st_sigmah,rt_sigmah,pair_num
,Float64,Float64,Float64,Float64,Float64,Float64,Int64
1,0.0,0.0,NaN,0.1,NaN,0.2,1
2,0.0,0.0,NaN,0.2,NaN,0.2,2
3,0.0,0.0,NaN,0.3,NaN,0.2,3
4,0.0,0.0,NaN,0.5,NaN,0.2,4
5,0.0,0.0,NaN,0.75,NaN,0.2,5
6,0.0,0.0,NaN,0.1,NaN,0.225,6
7,0.0,0.0,NaN,0.2,NaN,0.225,7
8,0.0,0.0,NaN,0.3,NaN,0.225,8
9,0.0,0.0,NaN,0.5,NaN,0.225,9


# Full Information

In [200]:
fieqdf

,vb,eq_negative,alpha,p,debt,c,eq_deriv,delta,pi,lambda,gross_delta,V0,nrm_lambda,kappa,datetime,rm_iota,iota,sigmal,eq_deriv_min_val,equity,mu_b,MBR,xi,eq_type,r,nrm_sigmah,m,eq_vb,leverage,type,sigmah,eq_min_val,rmp,firm_value
,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,DateTime,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Symbol,Float64,Float64,Float64,Float64,Float64,Symbol,Float64,Float64,Symbol,Float64
1,59.8202,0,0.6,54.0,56.4612,4.5,0.000226196,0.02,0.27,NaN,0.02,100.0,NaN,0.0025,2020-02-18T10:02:43.25,0.0,0.0,0.15,0.000226196,55.5114,1.04516,27.4987,1.0,full_info,0.08,NaN,1.0,0.0,50.4241,st,NaN,0.000284745,rm,111.973
2,50.884,0,0.6,54.0,46.1807,4.5,1.09135e-5,0.02,0.27,0.2,0.02,100.0,0.2,0.0025,2020-02-18T10:04:35.9,0.0,0.0,0.15,1.09135e-5,62.0365,0.854851,15.2682,1.0,full_info,0.08,0.225,1.0,0.0,42.6741,rt,0.225,0.0011387,nrm,108.217


In [245]:
uq_fidf = JointEqStructs.get_unique_df(jks_fpath, :fi; del_old_files=true)

Removing old files...
Saving filtered results...


,vb,eq_negative,alpha,p,debt,c,eq_deriv,delta,pi,lambda,gross_delta,V0,nrm_lambda,kappa,datetime,rm_iota,iota,sigmal,eq_deriv_min_val,equity,mu_b,MBR,xi,eq_type,r,nrm_sigmah,m,eq_vb,leverage,type,sigmah,eq_min_val,rmp,firm_value
,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,DateTime,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Symbol,Float64,Float64,Float64,Float64,Float64,Symbol,Float64,Float64,Symbol,Float64
1,59.8202,0,0.6,54.0,56.4612,4.5,0.000226196,0.02,0.27,NaN,0.02,100.0,NaN,0.0025,2020-02-18T10:02:43.25,0.0,0.0,0.15,0.000226196,55.5114,1.04516,27.4987,1.0,full_info,0.08,NaN,1.0,0.0,50.4241,st,NaN,0.000284745,rm,111.973
2,50.884,0,0.6,54.0,46.1807,4.5,1.09135e-5,0.02,0.27,0.2,0.02,100.0,0.2,0.0025,2020-02-18T10:04:35.9,0.0,0.0,0.15,1.09135e-5,62.0365,0.854851,15.2682,1.0,full_info,0.08,0.225,1.0,0.0,42.6741,rt,0.225,0.0011387,nrm,108.217


In [219]:
# cond  = .&(uq_fidf[:, :rmp] .== :nrm, uq_fidf[:, :type] .== :rt) .| (uq_fidf[:, :type] .== :st)
# fidf = deepcopy(uq_fidf[cond, :])
# CSV.write(fidf_fpath_name, fidf)

"/mnt/ide0/home/artur/BondPricing/Julia/Results/JEQ/kappa_25.00_bp__sigmal_0.150/m_1.00_pcr_12.00/zfidf.csv"

# Misrepresentation

In [201]:
misrepdf

,datetime,eq_type,kappa,mu_s,m,c,p,sf_defaults_first,misrep_type,s_fi_vb,st_vb,s_eq_deriv,s_eq_min_val,s_mu_b,s_eq_deriv_min_val,s_eq_negative,s_eq_vb,s_MBR,s_debt,s_equity,s_firm_value,s_leverage,s_iota,s_lambda,s_sigmah,s_delta,s_rmp,r_fi_vb,rt_vb,r_eq_deriv,r_eq_min_val,r_mu_b,r_eq_deriv_min_val,r_eq_negative,r_eq_vb,r_MBR,r_debt,r_equity,r_firm_value,r_leverage,r_iota,r_lambda,r_sigmah,r_delta,r_rmp,V0,alpha,pi,r,gross_delta,xi,sigmal,s_rm_iota,s_nrm_lambda,s_nrm_sigmah,r_rm_iota,r_nrm_lambda,r_nrm_sigmah
,DateTime,Symbol,Float64,Float64,Float64,Float64,Float64,Bool,Symbol,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Symbol,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Symbol,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2020-02-18T15:40:17.576,misrep,0.0025,1.0,1.0,4.5,54.0,0,rt,59.8202,59.8202,0.514906,0.318576,1.04516,0.514906,0,0.278834,27.4986,56.4612,55.5114,111.973,50.4241,0.0,NaN,NaN,0.02,rm,62.2119,60.8553,-0.0026039,0.00114346,1.04516,-0.0026039,0,0.0,17.4829,56.4612,51.1507,107.612,52.4674,0.0,0.2,0.225,0.02,nrm,100.0,0.6,0.27,0.08,0.02,1.0,0.15,0.0,NaN,NaN,0.0,0.2,0.225
2,2020-02-18T15:40:31.966,misrep,0.0025,0.0,1.0,4.5,54.0,0,st,48.9277,48.9277,0.34131,0.542494,0.854851,0.34131,0,0.520698,19.3204,46.1807,64.2174,110.398,41.8311,0.0,NaN,NaN,0.02,rm,50.8839,50.2366,1.40835e-5,0.000818788,0.854851,1.40835e-5,0,0.0,15.2848,46.1807,62.0455,108.226,42.6705,0.0,0.2,0.225,0.02,nrm,100.0,0.6,0.27,0.08,0.02,1.0,0.15,0.0,NaN,NaN,0.0,0.2,0.225


In [214]:
uq_misrepdf = JointEqStructs.get_unique_df(jks_fpath, :misrep; del_old_files=true)

Removing old files...
Saving filtered results...


,datetime,eq_type,kappa,mu_s,m,c,p,sf_defaults_first,misrep_type,s_fi_vb,st_vb,s_eq_deriv,s_eq_min_val,s_mu_b,s_eq_deriv_min_val,s_eq_negative,s_eq_vb,s_MBR,s_debt,s_equity,s_firm_value,s_leverage,s_iota,s_lambda,s_sigmah,s_delta,s_rmp,r_fi_vb,rt_vb,r_eq_deriv,r_eq_min_val,r_mu_b,r_eq_deriv_min_val,r_eq_negative,r_eq_vb,r_MBR,r_debt,r_equity,r_firm_value,r_leverage,r_iota,r_lambda,r_sigmah,r_delta,r_rmp,V0,alpha,pi,r,gross_delta,xi,sigmal,s_rm_iota,s_nrm_lambda,s_nrm_sigmah,r_rm_iota,r_nrm_lambda,r_nrm_sigmah
,DateTime,Symbol,Float64,Float64,Float64,Float64,Float64,Bool,Symbol,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Symbol,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Symbol,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2020-02-18T15:40:17.576,misrep,0.0025,1.0,1.0,4.5,54.0,0,rt,59.8202,59.8202,0.514906,0.318576,1.04516,0.514906,0,0.278834,27.4986,56.4612,55.5114,111.973,50.4241,0.0,NaN,NaN,0.02,rm,62.2119,60.8553,-0.0026039,0.00114346,1.04516,-0.0026039,0,0.0,17.4829,56.4612,51.1507,107.612,52.4674,0.0,0.2,0.225,0.02,nrm,100.0,0.6,0.27,0.08,0.02,1.0,0.15,0.0,NaN,NaN,0.0,0.2,0.225
2,2020-02-18T15:40:31.966,misrep,0.0025,0.0,1.0,4.5,54.0,0,st,48.9277,48.9277,0.34131,0.542494,0.854851,0.34131,0,0.520698,19.3204,46.1807,64.2174,110.398,41.8311,0.0,NaN,NaN,0.02,rm,50.8839,50.2366,1.40835e-5,0.000818788,0.854851,1.40835e-5,0,0.0,15.2848,46.1807,62.0455,108.226,42.6705,0.0,0.2,0.225,0.02,nrm,100.0,0.6,0.27,0.08,0.02,1.0,0.15,0.0,NaN,NaN,0.0,0.2,0.225


In [213]:
# cond  = .&(abs.(uq_misrepdf[:, :r_rm_iota]) .< 1e-5, uq_misrepdf[:, :r_rmp] .== :nrm)
# misrepdf = deepcopy(uq_misrepdf[cond, :])
# CSV.write(misrepdf_fpath_name, misrepdf)

"/mnt/ide0/home/artur/BondPricing/Julia/Results/JEQ/kappa_25.00_bp__sigmal_0.150/m_1.00_pcr_12.00/zmisrepdf.csv"

# Joint Equilibrium

In [257]:
for modl in modls
    include(string(joinpath(module_path, modl), ".jl"))
end

Figure(PyObject <Figure size 400x100 with 1 Axes>)

## Identify the Joint Equilibrium Case

In [221]:
jeqid

Dict{Symbol,Any} with 9 entries:
  :mu_b_grid           => [0.641138, 0.676155, 0.711172, 0.746188, 0.781205, 0.816221, 0.851238, 0.886254, 0.921271, 0.956287, 0.991304, 1.02632, 1.06134, 1.09635, 1.13137, 1.16639, 1.2014, 1.23642, 1.27144, 1.30645]
  :misrepdf_fpath_name => "/mnt/ide0/home/artur/BondPricing/Julia/Results/JEQ/kappa_25.00_bp__sigmal_0.150/m_1.00_pcr_12.00/zmisrepdf.csv"
  :fidf_fpath_name     => "/mnt/ide0/home/artur/BondPricing/Julia/Results/JEQ/kappa_25.00_bp__sigmal_0.150/m_1.00_pcr_12.00/zfidf.csv"
  :pooldf_fpath_name   => "/mnt/ide0/home/artur/BondPricing/Julia/Results/JEQ/kappa_25.00_bp__sigmal_0.150/m_1.00_pcr_12.00/zpooldf.csv"
  :fidf                => 2×34 DataFrame…
  :fieqdf              => 2×34 DataFrame…
  :misrepdf            => 2×58 DataFrame…
  :adf                 => 1×18 DataFrame…
  :sepdf_fpath_name    => "/mnt/ide0/home/artur/BondPricing/Julia/Results/JEQ/kappa_25.00_bp__sigmal_0.150/m_1.00_pcr_12.00/zsepdf.csv"

In [10]:
jeqid[:adf]

,m,c,p,misrep_type,st_rmp,rt_rmp,misrep_ic_objf,misrep_ic_objf_val,fi_st_mu_b,fi_st_vb,fi_rt_mu_b,fi_rt_vb,s_rm_iota,s_nrm_lambda,s_nrm_sigmah,r_rm_iota,r_nrm_lambda,r_nrm_sigmah
,Float64,Float64,Float64,Symbol,Symbol,Symbol,Symbol,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,4.5,54.0,rt,rm,nrm,MBR,15.2682,1.04516,59.8202,0.854851,50.884,0.0,NaN,NaN,0.0,0.2,0.225


## Compute Pooling Equilibrium Payoffs

In [ ]:
# pooldf = JointEq.get_jeq_results(jfep, jks, 
#                                 :pool, :exp_firm_value, 
#                                 jeqid; load_df=false,
#                                 recompute_df=true,
#                                 save_df=true)

In [23]:
uq_pooldf = JointEqStructs.get_unique_df(jks_fpath, :pool; del_old_files=true)

Removing old files...
Saving filtered results...


,datetime,eq_type,kappa,mu_s,m,c,p,sf_defaults_first,misrep_type,misrep_ic_objf,misrep_ic_objf_val,jeq_objf,s_fi_vb,st_vb,s_eq_deriv,s_eq_min_val,s_mu_b,s_eq_deriv_min_val,s_eq_negative,s_eq_vb,s_MBR,s_debt,s_equity,s_firm_value,s_leverage,s_iota,s_lambda,s_sigmah,s_delta,s_rmp,r_fi_vb,rt_vb,r_eq_deriv,r_eq_min_val,r_mu_b,r_eq_deriv_min_val,r_eq_negative,r_eq_vb,r_MBR,r_debt,r_equity,r_firm_value,r_leverage,r_iota,r_lambda,r_sigmah,r_delta,r_rmp,V0,alpha,pi,r,gross_delta,xi,sigmal,s_rm_iota,s_nrm_lambda,s_nrm_sigmah,r_rm_iota,r_nrm_lambda,r_nrm_sigmah
,DateTime,Symbol,Float64,Float64,Float64,Float64,Float64,Bool,Symbol,Symbol,Float64,Symbol,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Symbol,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Symbol,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2020-02-19T14:35:16.394,pool,0.0025,0.2,1.0,4.5,54.0,0,rt,MBR,15.2682,exp_firm_value,51.2285,51.2285,0.386588,0.459005,0.895051,0.386588,0,0.433209,20.9482,48.3524,62.4669,110.819,43.6317,0.0,NaN,NaN,0.02,rm,53.2768,52.4411,-0.000608258,0.000875894,0.895051,-0.000608258,0,0.0,15.9496,48.3524,59.8853,108.238,44.6724,0.0,0.2,0.225,0.02,nrm,100.0,0.6,0.27,0.08,0.02,1.0,0.15,0.0,NaN,NaN,0.0,0.2,0.225


In [22]:
pooldf = JointEq.get_jeq_results(jfep, jks, 
                                :pool, :exp_firm_value, 
                                jeqid; load_df=true,
                                recompute_df=false,
                                save_df=true)

,datetime,eq_type,kappa,mu_s,m,c,p,sf_defaults_first,misrep_type,misrep_ic_objf,misrep_ic_objf_val,jeq_objf,s_fi_vb,st_vb,s_eq_deriv,s_eq_min_val,s_mu_b,s_eq_deriv_min_val,s_eq_negative,s_eq_vb,s_MBR,s_debt,s_equity,s_firm_value,s_leverage,s_iota,s_lambda,s_sigmah,s_delta,s_rmp,r_fi_vb,rt_vb,r_eq_deriv,r_eq_min_val,r_mu_b,r_eq_deriv_min_val,r_eq_negative,r_eq_vb,r_MBR,r_debt,r_equity,r_firm_value,r_leverage,r_iota,r_lambda,r_sigmah,r_delta,r_rmp,V0,alpha,pi,r,gross_delta,xi,sigmal,s_rm_iota,s_nrm_lambda,s_nrm_sigmah,r_rm_iota,r_nrm_lambda,r_nrm_sigmah
,DateTime,Symbol,Float64,Float64,Float64,Float64,Float64,Bool,Symbol,Symbol,Float64,Symbol,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Symbol,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Symbol,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2020-02-19T14:35:16.394,pool,0.0025,0.2,1.0,4.5,54.0,0,rt,MBR,15.2682,exp_firm_value,51.2285,51.2285,0.386588,0.459005,0.895051,0.386588,0,0.433209,20.9482,48.3524,62.4669,110.819,43.6317,0.0,NaN,NaN,0.02,rm,53.2768,52.4411,-0.000608258,0.000875894,0.895051,-0.000608258,0,0.0,15.9496,48.3524,59.8853,108.238,44.6724,0.0,0.2,0.225,0.02,nrm,100.0,0.6,0.27,0.08,0.02,1.0,0.15,0.0,NaN,NaN,0.0,0.2,0.225


## Compute Separating Equilibrium Payoffs

In [17]:
for modl in modls
    include(string(joinpath(module_path, modl), ".jl"))
end

Figure(PyObject <Figure size 400x100 with 1 Axes>)

In [27]:
# sepdf = JointEq.get_jeq_results(jfep, jks, 
#                                 :sep, :st_firm_value, 
#                                 jeqid; load_df=false,
#                                 recompute_df=true,
#                                 save_df=true)

Computing Equity Vmax
Vmax: 125.0
phi0: -25.980581645833045
phi1: 0.9999999999999998
pv_rfdebt: 34.6355116681809
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
debt_abs_per_diff: 5.542605098344562e-6
eq_abs_per_diff: 0.0009420896842749746
Equity Vmax: 235.70614279040458
 
eq_m

V0: 100.0; equity: 72.83186172625602
Equity Core Function Computation Time: 0.056633949279785156
Total Equity FD Core Function Computation Time: 0.1055748462677002
Total computation time: 0.13212895393371582
Computing Equity Vmax
Vmax: 125.0
phi0: -25.980581645833045
phi1: 0.9999999999999998
pv_rfdebt: 34.6355116681809
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.641138420

V0: 100.0; equity: 72.9287949387657
Equity Core Function Computation Time: 0.05520915985107422
Total Equity FD Core Function Computation Time: 0.10235714912414551
Total computation time: 0.12867999076843262
Computing Equity Vmax
Vmax: 125.0
phi0: -25.980581645833045
phi1: 0.9999999999999998
pv_rfdebt: 34.6355116681809
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205

V0: 100.0; equity: 72.91014461345084
Equity Core Function Computation Time: 0.08329987525939941
Total Equity FD Core Function Computation Time: 0.13723206520080566
Total computation time: 0.16438984870910645
Computing Equity Vmax
Vmax: 125.0
phi0: -25.980581645833045
phi1: 0.9999999999999998
pv_rfdebt: 34.6355116681809
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.641138420

 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
debt_abs_per_diff: 5.542605098344562e-6
eq_abs_per_diff: 0.0009420896842749746
Equity Vmax: 235.70614279040458
 
eq_max: 1.9464433937140264
eq_vbl: 0.0
eq_vbl: 0.0
 mu_b = 0.6411384205417969
 m = 1.0
 c = 4.5
 p = 54.0
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 73.96983992553389
Equity Core Function Computation Time: 0.06804800033569336
Total Equity FD Core Function Computation Time: 0.10413599014282227
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.008751153945922852
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 72.92797683100406
Equity Core Function Computation Time: 0.08327198028564453
Total Equity FD Core Function Computation Time: 0.1392529010772705
Total computation time: 0.28050994873046875
  
vbl: 34.04911833820205
  
Computing Equity Vmax
Vmax: 125.0
phi

...

V0: 100.0; equity: 69.56269067991914
Equity Core Function Computation Time: 0.13930511474609375
Total Equity FD Core Function Computation Time: 0.1752490997314453
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.008800983428955078
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 67.86756715948377
Equity Core Function Computation Time: 0.23900890350341797
Total Equity FD Core Function Computation Time: 0.2960078716278076
Total computation time: 0.5113229751586914
  
vbl: 42.27641982841808
  
Computing Equity Vmax
Vmax: 125.0
phi0: -30.23746449430697
phi1: 0.9999999999999998
pv_rfdebt: 40.310492989934524
 mu_b = 0.7461880758230818
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.7461880758230818
 m = 1.0
 c = 4.5
 p = 54.0
debt_abs_per_diff: NaN
eq_abs_per_diff: NaN
Vmax: 125.0
phi0: -30.23746449430697
phi1: 0.9999999999999998
pv_rfdebt: 40.310492989934524
 mu_b = 0.74

V0: 100.0; equity: 69.48757073440451
Equity Core Function Computation Time: 0.04874396324157715
Total Equity FD Core Function Computation Time: 0.08489298820495605
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.008842945098876953
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
V0: 100.0; equity: 67.85367140281483
Equity Core Function Computation Time: 0.04670405387878418
Total Equity FD Core Function Computation Time: 0.0988149642944336
Total computation time: 0.22418808937072754
  
vbl: 44.47010531587898
  
Computing Equity Vmax
Vmax: 125.0
phi0: -30.23746449430697
phi1: 0.9999999999999998
pv_rfdebt: 40.310492989934524
 mu_b = 0.7461880758230818
 m = 1.0
 c = 4.5
 p = 54.0
 mu_b = 0.7461880758230818
 m = 1.0
 c = 4.5
 p = 54.0
debt_abs_per_diff: NaN
eq_abs_per_diff: NaN
Vmax: 125.0
phi0: -30.23746449430697
phi1: 0.9999999999999998
pv_rfdebt: 40.310492989934524
 mu_b = 0.

Excessive output truncated after 524298 bytes.

V0: 45.01852668774421; equity: 0.11388153519550308
Equity Core Function Computation Time: 0.05402708053588867
Total Equity FD Core Function Computation Time: 0.09496903419494629
Total computation time: 0.12440013885498047


,datetime,eq_type,kappa,mu_s,m,c,p,sf_defaults_first,misrep_type,misrep_ic_objf,misrep_ic_objf_val,jeq_objf,s_MBR,s_debt,s_delta,s_eq_deriv,s_eq_deriv_min_val,s_eq_min_val,s_eq_negative,s_eq_vb,s_equity,s_firm_value,s_iota,s_lambda,s_leverage,s_mu_b,s_sigmah,s_fi_vb,st_vb,s_rmp,r_MBR,r_debt,r_delta,r_eq_deriv,r_eq_deriv_min_val,r_eq_min_val,r_eq_negative,r_eq_vb,r_equity,r_firm_value,r_iota,r_lambda,r_leverage,r_mu_b,r_sigmah,r_fi_vb,rt_vb,r_rmp,V0,alpha,pi,r,gross_delta,xi,sigmal,s_rm_iota,s_nrm_lambda,s_nrm_sigmah,r_rm_iota,r_nrm_lambda,r_nrm_sigmah
,DateTime,Symbol,Float64,Float64,Float64,Float64,Float64,Bool,Symbol,Symbol,Float64,Symbol,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Symbol,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Symbol,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2020-02-19T15:56:02.761,sep,0.0025,0.2,1.0,4.5,54.0,1,rt,MBR,15.2682,st_firm_value,30.8759,64.3026,0.02,0.000223679,0.000223679,0.000226821,0,0.0,46.7192,111.022,0.0,NaN,57.9189,1.19065,NaN,68.147,68.147,rm,15.2682,46.1807,0.02,1.09135e-5,1.09135e-5,0.0011387,0,0.0,62.0365,108.217,0.0,0.2,42.6741,0.854851,0.225,50.884,50.884,nrm,100.0,0.6,0.27,0.08,0.02,1.0,0.15,0.0,NaN,NaN,0.0,0.2,0.225


In [28]:
uq_sepdf = JointEqStructs.get_unique_df(jks_fpath, :sep;
                                        save_filtered_df=true,
                                        del_old_files=true)

Removing old files...
Saving filtered results...


,datetime,eq_type,kappa,mu_s,m,c,p,sf_defaults_first,misrep_type,misrep_ic_objf,misrep_ic_objf_val,jeq_objf,s_MBR,s_debt,s_delta,s_eq_deriv,s_eq_deriv_min_val,s_eq_min_val,s_eq_negative,s_eq_vb,s_equity,s_firm_value,s_iota,s_lambda,s_leverage,s_mu_b,s_sigmah,s_fi_vb,st_vb,s_rmp,r_MBR,r_debt,r_delta,r_eq_deriv,r_eq_deriv_min_val,r_eq_min_val,r_eq_negative,r_eq_vb,r_equity,r_firm_value,r_iota,r_lambda,r_leverage,r_mu_b,r_sigmah,r_fi_vb,rt_vb,r_rmp,V0,alpha,pi,r,gross_delta,xi,sigmal,s_rm_iota,s_nrm_lambda,s_nrm_sigmah,r_rm_iota,r_nrm_lambda,r_nrm_sigmah
,DateTime,Symbol,Float64,Float64,Float64,Float64,Float64,Bool,Symbol,Symbol,Float64,Symbol,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Symbol,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Symbol,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2020-02-19T15:56:02.761,sep,0.0025,0.2,1.0,4.5,54.0,1,rt,MBR,15.2682,st_firm_value,30.8759,64.3026,0.02,0.000223679,0.000223679,0.000226821,0,0.0,46.7192,111.022,0.0,NaN,57.9189,1.19065,NaN,68.147,68.147,rm,15.2682,46.1807,0.02,1.09135e-5,1.09135e-5,0.0011387,0,0.0,62.0365,108.217,0.0,0.2,42.6741,0.854851,0.225,50.884,50.884,nrm,100.0,0.6,0.27,0.08,0.02,1.0,0.15,0.0,NaN,NaN,0.0,0.2,0.225


In [29]:
sepdf = JointEq.get_jeq_results(jfep, jks, 
                                :sep, :st_firm_value, 
                                jeqid; load_df=true,
                                recompute_df=false,
                                save_df=true)

,datetime,eq_type,kappa,mu_s,m,c,p,sf_defaults_first,misrep_type,misrep_ic_objf,misrep_ic_objf_val,jeq_objf,s_MBR,s_debt,s_delta,s_eq_deriv,s_eq_deriv_min_val,s_eq_min_val,s_eq_negative,s_eq_vb,s_equity,s_firm_value,s_iota,s_lambda,s_leverage,s_mu_b,s_sigmah,s_fi_vb,st_vb,s_rmp,r_MBR,r_debt,r_delta,r_eq_deriv,r_eq_deriv_min_val,r_eq_min_val,r_eq_negative,r_eq_vb,r_equity,r_firm_value,r_iota,r_lambda,r_leverage,r_mu_b,r_sigmah,r_fi_vb,rt_vb,r_rmp,V0,alpha,pi,r,gross_delta,xi,sigmal,s_rm_iota,s_nrm_lambda,s_nrm_sigmah,r_rm_iota,r_nrm_lambda,r_nrm_sigmah
,DateTime,Symbol,Float64,Float64,Float64,Float64,Float64,Bool,Symbol,Symbol,Float64,Symbol,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Symbol,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Symbol,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,2020-02-19T15:56:02.761,sep,0.0025,0.2,1.0,4.5,54.0,1,rt,MBR,15.2682,st_firm_value,30.8759,64.3026,0.02,0.000223679,0.000223679,0.000226821,0,0.0,46.7192,111.022,0.0,NaN,57.9189,1.19065,NaN,68.147,68.147,rm,15.2682,46.1807,0.02,1.09135e-5,1.09135e-5,0.0011387,0,0.0,62.0365,108.217,0.0,0.2,42.6741,0.854851,0.225,50.884,50.884,nrm,100.0,0.6,0.27,0.08,0.02,1.0,0.15,0.0,NaN,NaN,0.0,0.2,0.225


In [ ]:
HEEEEEREEEEEEEEE

In [37]:
for modl in modls
    include(string(joinpath(module_path, modl), ".jl"))
end

Figure(PyObject <Figure size 400x100 with 1 Axes>)

In [15]:
# Make Directories and File Names ##########################
# Make results directories
jks_fpath = JointEq.make_jeq_jks_fpath(jfep)

# Full Information Equilibrium
fi_fpath_name = JointEq.get_jeq_mus_fname(jks_fpath; eq_type="full_info")

# Misrepresentation
misrep_fpath_name = JointEq.get_jeq_mus_fname(jks_fpath; eq_type="misrep")

# Pooling Equilibrium
pool_fpath_name = JointEq.get_jeq_mus_fname(jks_fpath; eq_type="pooling", mu_s=jks.mu_s)

# Separating Equilibrium
sep_fpath_name = JointEq.get_jeq_mus_fname(jks_fpath; eq_type="separating")
# #########################################################



"/mnt/ide0/home/artur/BondPricing/Julia/Results/JEQ/kappa_25.00_bp__sigmal_0.150/m_1.00_pcr_12.00/sep_tmp.csv"

In [ ]:
function ep_constructor(jf;
                        run_misrep::Bool=false,
                        run_pool_eq::Bool=true,
                        run_sep_eq::Bool=true,                       
                        sf_obj_fun::Symbol=:firm_value,
                        rf_obj_fun::Symbol=:MBR,
                        fi_fpath_name::String="",
                        rerun_full_info::Bool=true,
                        rerun_pool::Bool=true,
                        rerun_sep::Bool=true,
                        lb::Float64=.75,
                        ub::Float64=1.25,
                        mu_bN::Int64=20,
                        mu_bN2::Int64=10^5,
                        spline_k::Int64=3,
                        spline_bc::String="extrapolate")

    # Full Information Equilibrium
    fidf, fieqdf = FullInfoEq.get_fi_eq(jfep)
    
    # Extract Firms and set joint capital structure
    st, rt, ep_jks = JointEqStructs.form_firms_jks(jfep, fieqdf)
    
    # Check for missing parameters
    if any([isnan(getfield(ep_jks, x)) for x in [:mu_s, :m, :c, :p]])
        println("Missing Electronic Platform parameters")
        return
    end

    # Electronic Platform Full-Information Equilibrium ################
    if .&(isfile(fi_fpath_name), !rerun_full_info)
        fidf = CSV.read(fi_fpath_name)
                        #types=fidf_col_types)
        _, fieqdf = FullInfoEq.get_fi_eq(jfep, fidf=fidf)

        # Capture Full Information Optimal mu_b and MBR ###################
        fi_sf_mu_b = fieqdf[fieqdf[:, :type] .== :st, :mu_b][1]
        fi_rf_mu_b = fieqdf[fieqdf[:, :type] .== :rt, :mu_b][1]
        fi_sf_obj_val = fieqdf[fieqdf[:, :type] .== :st, sf_obj_fun][1]
        fi_rf_obj_val = fieqdf[fieqdf[:, :type] .== :st, sf_obj_fun][1]
        # #################################################################
    elseif rerun_full_info
        fidf, fieqdf = FullInfoEq.get_fi_eq(jfep)

        ep_sf_eqdf = find_optimal_bond_measure(ep_sf_svm; jks=ep_jks)
        ep_rf_eqdf = find_optimal_bond_measure(ep_rf_svm; jks=ep_jks)

        # Capture Full Information Optimal mu_b and MBR ###################
        fi_sf_mu_b = fieqdf[fieqdf[:, :type] .== :st, :mu_b][1]
        fi_rf_mu_b = fieqdf[fieqdf[:, :type] .== :rt, :mu_b][1]
        fi_sf_obj_val = fieqdf[fieqdf[:, :type] .== :st, sf_obj_fun][1]
        fi_rf_obj_val = fieqdf[fieqdf[:, :type] .== :st, sf_obj_fun][1]
        # #################################################################
    else
        fidf = DataFrame()
        fieqdf = DataFrame()

        fi_sf_mu_b = NaN
        fi_rf_mu_b =  NaN
        fi_sf_obj_val =  NaN
        fi_rf_obj_val =  NaN

        run_pool_eq = false
        run_sep_eq = false
    end
    # #################################################################

            
    # Electronic Platform Misrepresentation ###########################
    # Do risky firms have an incentive to copy the capital structure
    # of the safe firms?
    if run_misrep
        misrep_jks = deepcopy(ep_jks)
        setfield!(misrep_jks, :mu_s, 1.)
        if !isnan(fi_sf_mu_b)
            setfield!(misrep_jks, :mu_b, fi_sf_mu_b)
        end
        
        ep_misrep_eqdf = find_joint_optimal_vb(ep_jf, misrep_jks;
                                               mu_b=fi_sf_mu_b,
                                               rerun_fi_vb=true)

        # Add Objective Function columns
        ep_misrep_eqdf[!, :obj_fun] .= String(sf_obj_fun)
        ep_misrep_eqdf[2, :obj_fun] = "misrep"
        ep_misrep_eqdf[!, :eq_type] .= "misrep"
        
        # Reshape
        ep_misrep_eqdf = reshape_sf_rf_df(ep_misrep_eqdf)
    else
        ep_misrep_eqdf = DataFrame(mp)
    end
    # ##################################################################


    # Electronic Platform Pooling and Separating Equilibria ############
    eq_type = "all"
    run_pool_sep_eq = true
    if .&(run_pool_eq, !run_sep_eq)
        eq_type = "pooling"
    elseif .&(!run_pool_eq, run_sep_eq)
        eq_type = "separating"
    elseif .&(!run_pool_eq, !run_sep_eq)
        run_pool_sep_eq = false
    end

    ep_pool_eqdf = DataFrame()
    ep_sep_eqdf = DataFrame()
    if run_pool_sep_eq
        ep_eqdf = ep_pool_sep_eq(ep_jf, deepcopy(ep_jks),
                                 fi_sf_mu_b,
                                 fi_rf_mu_b,
                                 fi_rf_obj_val;
                                 equilibrium_type=eq_type,
                                 sf_obj_fun=sf_obj_fun,
                                 rf_obj_fun=rf_obj_fun,
                                 rerun=true,
                                 lb=lb, ub=ub,
                                 mu_bN=mu_bN, mu_bN2=mu_bN2,
                                 spline_k=spline_k,
                                 spline_bc=spline_bc)

        if ("pooling" in ep_eqdf[:, :eq_type])
            ep_pool_eqdf = ep_eqdf[ep_eqdf[:, :eq_type] .== "pooling", :]
        end
        if ("separating" in ep_eqdf[:, :eq_type])
            ep_sep_eqdf = ep_eqdf[ep_eqdf[:, :eq_type] .== "separating", :]
        end
    end
    # #################################################################


   # Form Electronic Platform Struct #################################
   return EPStruct(ep_jf, ep_sf_eqdf, ep_rf_eqdf,
                   ep_misrep_eqdf, ep_pool_eqdf, ep_sep_eqdf)
end

